In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

!pip install metric-learn

from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from metric_learn import MMC_Supervised

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 819.9 kB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import h5py

houston18_file = h5py.File('/content/drive/MyDrive/proje/Houston18.mat', 'r')
houston18_gt_file = h5py.File('/content/drive/MyDrive/proje/Houston18_7gt.mat', 'r')

data = np.array(houston18_file['ori_data'])
gt = np.array(houston18_gt_file['map'])


def applyPCA(X, numComponents=15):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX


data = np.transpose(data, (1, 2, 0))

data=applyPCA(data)

df = pd.DataFrame(data.reshape(data.shape[0]*data.shape[1], -1))
df['class'] = gt.ravel()

X = df[df['class']!=0].iloc[:, :-1].values

y = df[df['class']!=0].iloc[:, -1].values



In [4]:


acc_list = []

kf = KFold(n_splits=5, shuffle=True,random_state=42)

mmc = MMC_Supervised(n_constraints=5,verbose=True)
knn = KNeighborsClassifier(n_neighbors=5)

f_index = 1
for train_index, test_index in kf.split(X):
    print(f"Fold {f_index} -> Train: {len(train_index)}, Test: {len(test_index)}")

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    mmc.fit(X_train, y_train)
    knn.fit(mmc.transform(X_train), y_train)


    mmc_acc = knn.score(mmc.transform(X_test), y_test)
    print('mmc accuracy  {}'.format(mmc_acc))

    acc_list.append(mmc_acc)

    f_index += 1

print("Ortalaması Başarı:", np.mean(acc_list))
print("Standart sapma:",np.std(acc_list) )

Fold 1 -> Train: 42560, Test: 10640
mmc iter: 0, conv = 0.033108, projections = 1096
mmc iter: 1, conv = 0.034490, projections = 280
mmc iter: 2, conv = 0.035898, projections = 352
mmc iter: 3, conv = 0.037328, projections = 366
mmc iter: 4, conv = 0.038778, projections = 363
mmc iter: 5, conv = 0.040246, projections = 356
mmc iter: 6, conv = 0.041738, projections = 547
mmc iter: 7, conv = 0.043250, projections = 573
mmc iter: 8, conv = 0.044777, projections = 579
mmc iter: 9, conv = 0.046316, projections = 584
mmc iter: 10, conv = 0.047864, projections = 588
mmc iter: 11, conv = 0.049417, projections = 592
mmc iter: 12, conv = 0.050970, projections = 596
mmc iter: 13, conv = 0.052520, projections = 600
mmc iter: 14, conv = 0.054063, projections = 604
mmc iter: 15, conv = 0.055593, projections = 611
mmc iter: 16, conv = 0.057116, projections = 845
mmc iter: 17, conv = 0.058646, projections = 1083
mmc iter: 18, conv = 0.060185, projections = 1283
mmc iter: 19, conv = 0.061732, projectio

In [5]:
import joblib

joblib.dump(mmc, 'mmc_houston_model.pkl')
joblib.dump(knn,'mmc_houston_knn_model.pkl')

mmc_loaded = joblib.load('mmc_houston_model.pkl')
knn_loaded =joblib.load('mmc_houston_knn_model.pkl')

mmc_data=mmc_loaded.transform(X)
pred=knn_loaded.predict(mmc_data)

In [6]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

print("accuracy:",accuracy_score(pred,y))
print("cohen kappa:", cohen_kappa_score(pred,y))
print("f1 score:",f1_score(pred,y,average='macro'))

accuracy: 0.9110150375939849
cohen kappa: 0.851079845496812
f1 score: 0.8807343230573437


In [7]:
gt_2 = gt.copy()
gt_2=gt_2.reshape(-1)


i = 0
j=0
while(i<(gt.shape[0]*gt.shape[1])):
      if(gt_2[i] != 0):
          gt_2[i]=pred[j]
          j += 1
      i += 1

In [8]:
gt_2=gt_2.reshape(data.shape[0],-1)

In [ ]:
def plot_gt(data):
  fig = plt.figure(figsize=(12, 8))
  plt.imshow(data,cmap='nipy_spectral')
  plt.colorbar()
  plt.axis('off')
  plt.show()

plot_gt(gt)

In [ ]:
plot_gt(gt_2)